In [1]:
import os
from random import sample
import pandas as pd
from eyened_orm import ImageInstance, SourceInfo, DBManager
from eyened_orm.utils import get_config

# from rtnls_fundusprep.preprocessor import FundusPreprocessor
# from rtnls_fundusprep.utils import preprocess_for_inference
from sqlalchemy import select, func
from tqdm.notebook import tqdm
import pydicom
from pathlib import Path

In [3]:
from random import sample
from PIL import Image
import numpy as np
import cv2
from eyened_orm.importer.thumbnails import get_thumbnail, save_thumbnails

ImportError: cannot import name 'get_thumbnail' from 'eyened_orm.importer.thumbnails' (/home/jeroen/Git/eyened-platform/orm/eyened_orm/importer/thumbnails.py)

In [3]:
config = get_config("prod")
DBManager.init(config)
session = DBManager.get_session()

In [4]:

stmt = select(ImageInstance).where(ImageInstance.DatasetIdentifier.like(f"%.binary"))
images = session.execute(stmt).scalars().all()

In [4]:
images = session.execute(select(ImageInstance).order_by(func.random()).limit(2000)).scalars().all()

In [5]:
images_binary = [im for im in images if im.path.endswith(".binary")]
images_dicom = [im for im in images if im.path.endswith(".dcm")]
images_png = [im for im in images if im.path.endswith(".png")]

In [6]:
def plot(ims_sample):
    import numpy as np

    import matplotlib.pyplot as plt
    
    fig, axs = plt.subplots(2, 4, figsize=(15, 8))
    axs = axs.flatten()

    for i, (im) in enumerate(ims_sample):
        print(im.ImageInstanceID, im.path)
        # try:
        # Use the read_thumbnail function defined earlier to get the image
        if im.path.endswith(".json"):
            continue
        img = get_thumbnail(im)
        # print(img)
        if img is not None:
            if img.ndim == 3:
                axs[i].imshow(img)
            else:
                axs[i].imshow(img, cmap="gray")
        # except Exception as e:
            # print(f'Error: {str(e)}')
        
        axs[i].set_title(f"ID: {im.ImageInstanceID}")
        axs[i].axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
ims_sample = sample(images_png, 8)
plot(ims_sample)

In [ ]:
ims_sample = sample(images_dicom, 8)
plot(ims_sample)

In [ ]:
ims_sample = sample(images_binary, 8)
plot(ims_sample)